#  Capstone Project - The Battle of Neighborhoods

### Best Places for Coffee Shop in Istanbul!

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
import pandas as pd
import folium

In [4]:
CLIENT_ID = 'HCIWEYMLE0SJAI3ESV4AFX5PNQVBSLP5HQ1YU4GISAHHRIFV' # your Foursquare ID
CLIENT_SECRET = 'P4KVBEVJDIVREULUPIZHUL124JX353PUIP5KWJOGX1PLDB5B' # your Foursquare Secret
VERSION = '20200202' # Foursquare API version

In [8]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "İstanbul",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [9]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [10]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Istanbul', 'Istanbul')

In [11]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 41.021645123314876, 'lng': 28.961753664589285},
  'sw': {'lat': 41.007660082677425, 'lng': 28.93980187306155}},
 199)

In [12]:
d["geocode"]

{'what': '',
 'where': 'istanbul',
 'center': {'lat': 41.01384, 'lng': 28.94966},
 'displayString': 'Istanbul, Turkey',
 'cc': 'TR',
 'geometry': {'bounds': {'ne': {'lat': 41.583896981907635,
    'lng': 29.95880097737308},
   'sw': {'lat': 40.80260400000006, 'lng': 27.971700000085676}}},
 'slug': 'istanbul',
 'longId': '72057594038672980'}

In [13]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [14]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [15]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '548ed3d0498e646a6e79463d',
  'name': 'Best Kahve',
  'location': {'address': 'Aksaray',
   'crossStreet': 'İskenderpaşa Mh.',
   'lat': 41.01269760481526,
   'lng': 28.94727581257156,
   'labeledLatLngs': [{'label': 'display',
     'lat': 41.01269760481526,
     'lng': 28.94727581257156}],
   'cc': 'TR',
   'city': 'İstanbul',
   'state': 'Vatan Cd.',
   'country': 'Türkiye',
   'formattedAddress': ['Aksaray (İskenderpaşa Mh.)', 'İstanbul', 'Türkiye']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-548ed3d0498e646a6e79463d-0'}

In [16]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5a43b13b340a583905a52551',
  'name': 'Safir Lounge',
  'location': {'address': 'Ali Kuşçu Mahallesi Hulusi Noyan Sk. No:3 Fatih İstanbul',
   'crossStreet': 'Macar Kardeşler Caddesi',
   'lat': 41.0170961145449,
   'lng': 28.950641953225276,
   'labeledLatLngs': [{'label': 'display',
     'lat': 41.0170961145449,
     'lng': 28.950641953225276}],
   'postalCode': '34083',
   'cc': 'TR',
   'neighborhood': 'Ali Kuscu',
   'city': 'İstanbul',
   'state': 'İstanbul',
   'country': 'Türkiye',
   'formattedAddress': ['Ali Kuşçu Mahallesi Hulusi Noyan Sk. No:3 Fatih İstanbul (Macar Kardeşler Caddesi)',
    '34083 İstanbul',
    'Türkiye']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caf

In [18]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["formattedAddress"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "formattedAddress", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 14 cafes


,uid,name,shortname,formattedAddress,postalcode,lat,lng
0,5a43b13b340a583905a52551,Safir Lounge,Café,[Ali Kuşçu Mahallesi Hulusi Noyan Sk. No:3 Fat...,34083,41.017096,28.950642
1,586fbf539343e02196adffb9,Şam Kahvesi,Café,"[Ali Kuşçu Mahallesi, Taylasan Sokak No: 8/A (...",34083,41.017899,28.952020
2,581069fc38fa62276eadd637,Rumeli Çikolatacısı,Chocolate Shop,[Fevzi Paşa Caddesi No:13 (Fevzi paşa caddesi)...,34080,41.017599,28.949107
3,5925c60c396de00b7278eb19,EspressoLab,Coffee Shop,"[Fevzi Paşa Cad. No: 17 (Fevzi Paşa Cad.), 340...",34080,41.017388,28.949536
4,585d4cc045005e7adc4239ad,Sütlü Kahve,Café,"[Fevzipaşa Caddesi, Fatih, 34080 İstanbul, Tür...",34080,41.019022,28.947946


In [20]:
istanbul_center = d["geocode"]["center"]
istanbul_center

{'lat': 41.01384, 'lng': 28.94966}

In [21]:
from folium import plugins

# create map of Istanbul using latitude and longitude values
map_istanbul = folium.Map(location=[istanbul_center["lat"], istanbul_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_istanbul)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_istanbul.add_child(plugins.HeatMap(hm_data))

map_istanbul

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [24]:
lat = 41.01384 
lng = 28.94966
map_istanbul = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_istanbul)
map_istanbul

In [ ]:
As you can see, one possibly good location for new Cafe would be in crossroad of Sofular Caddesi.